In [26]:
import nltk
import sklearn
import pandas
import numpy as np
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.classify import MaxentClassifier
from nltk import classify
from sklearn import cross_validation
import re
import codecs
import freeling

ImportError: No module named 'freeling'

### 1. Importamos el corpus de comentarios

In [10]:
comentarios = pandas.read_csv("comentarios_peliculas.csv", skiprows=1, delimiter=';', skip_blank_lines=True, encoding='utf-8')

### 1.1 Realizamos un breve analisis del corpus 

In [11]:

positivos = 0 
neutros = 0
negativos = 0
total = str(len(comentarios))
largo_comentario = 0
lista_comentarios = comentarios.values.tolist()
for x in lista_comentarios:
    if x[7] < 3:
        negativos += 1
    elif x[7] == 3:
        neutros += 1
    else:
        positivos += 1
    largo_comentario += len(re.findall(r'\w+', x[6]))


avg_positivos = str(float(positivos)*100 / float(total))
avg_negativos = str(float(negativos)*100 / float(total))
avg_neutros = str(float(neutros)*100 / float(total))
avg_largo_comentario = str(float(largo_comentario)/ float(total))
        
print ("El corpus tiene " + total + " comentarios")
print ("Hay " + str(positivos) + " comentarios positivos lo que equivale al  " + avg_positivos[:4] + "%")
print ("Hay " + str(neutros) + " comentarios neutros lo que equivale al  " + avg_neutros[:4] + "%")
print ("Hay " + str(negativos) + " comentarios negativos lo que equivale al  " + avg_negativos[:4] + "%")
print ("El largo de palabras promedio de los comentarios es " + avg_largo_comentario[:2])

El corpus tiene 1447 comentarios
Hay 859 comentarios positivos lo que equivale al  59.3%
Hay 222 comentarios neutros lo que equivale al  15.3%
Hay 366 comentarios negativos lo que equivale al  25.2%
El largo de palabras promedio de los comentarios es 38


### 2. Separamos entrenamiento y testeo, lo pasamos a listas comunes y eliminamos datos que no utilizamos. 

In [58]:
np.random.seed(42) 
msk = np.random.rand(len(comentarios)) < 0.8
comentarios_training= comentarios[msk] 
comentarios_testing = comentarios[~msk] 

lista_comentarios_training = comentarios_training.values.tolist()
for x in lista_comentarios_training:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de entrenamiento tiene largo " + str(len(lista_comentarios_training)))
lista_comentarios_testing = comentarios_testing.values.tolist()
for x in lista_comentarios_testing:
    del x[8]
    del x[5]
    del x[4]
    del x[3]
    del x[2]
    del x[1]
    del x[0]
print ("El corpus de testing tiene largo " + str(len(lista_comentarios_testing)))

El corpus de entrenamiento tiene largo 1147
El corpus de testing tiene largo 300


### 3. Obtenemos las palabras mas comunes del corpus, quitando puntuación y stopwords del Español

In [59]:
lista_comentarios_training_token = []
for comentario in lista_comentarios_training:
    c = tokenizer.tokenize(comentario[0].lower())
    lista_comentarios_training_token.append((c,comentario[1]))

textoComentariosTraining = ""
for x in lista_comentarios_training_token:
    for l in x[0]:
        textoComentariosTraining += " " + l

tokenizer = RegexpTokenizer(r'\w+')
corpus_text = tokenizer.tokenize(textoComentariosTraining)
stopwordsEspanol = nltk.corpus.stopwords.words('spanish')
stopwordsEspanol.remove("no")
stopwordsEspanol.remove("sí")
palabrasMasComunes = FreqDist(w.lower() for w in corpus_text if w not in stopwordsEspanol)

### 4. Utilizamos cross-validation para determinar el mejor N

In [64]:
def word_feats(commentWords): # FUNCION QUE DEVUELVE UN DICCIONARIO DE FEATURES, CON VALORES TRUE Y FALSE
    return dict([(word[0], word[0] in commentWords) for word in palabras_features])

def classification(value): # FUNCION QUE TRANSFORMA UN NUMERO DE 1..5 A UNA CLASIFICACION
    if value > 3:
        classification = "negative"
    elif value == 3:
        classification = "neutro"
    else:
        classification = "positive"
    return classification

def crossValidation(array): # Funcion que calcula la accuracy dado cierto array de entrenamiento, 
                            # que es un array, de array de features y su clasificacion
    k = 10 # Por defecto, se utiliza 10
    max_iterations = 40 
    totalAccuracy = 0
    cv = cross_validation.KFold(len(array), k, None, False)
    for traincv, testcv in cv: 
        classifier = nltk.MaxentClassifier.train(array[traincv[0]:traincv[len(traincv)-1]], max_iter=max_iterations) 
        accuracyCV = nltk.classify.util.accuracy(classifier, array[testcv[0]:testcv[len(testcv)-1]])
        print ('accuracy:', accuracyCV)
        totalAccuracy += accuracyCV
    return (totalAccuracy/k)

# VARIABLES DEL FOR. DETERMINAN EL TIEMPO DE EJECUCION EN GRAN MEDIDA
posiblesN = [40] 

listaAccuracy = [] 

for n in posiblesN:
    palabras_features = palabrasMasComunes.most_common(n)
    array_for_training = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_training_token]
    accuracyN = crossValidation(array_for_training)
    listaAccuracy.append(accuracyN)
    
print(listaAccuracy)

# SELECCIONAMOS EL MEJOR N
index = listaAccuracy.index(max(listaAccuracy)) # OBTENGO INDICE DEL MAYOR
N = posiblesN[index]

#para ver la accuracy contra testing, cosa que NO hay que hacer:::
#array_for_test = [(word_feats(f[0]), classification(f[1])) for f in lista_comentarios_testing]
#accuracyTesting = classify.accuracy(bestClassifierCV,array_for_test)

  ==> Training (40 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.255
             2          -0.81430        0.589
             3          -0.80195        0.589
             4          -0.79017        0.589
             5          -0.77896        0.589
             6          -0.76832        0.590
             7          -0.75824        0.596
             8          -0.74869        0.603
             9          -0.73966        0.607
            10          -0.73113        0.615
            11          -0.72307        0.629
            12          -0.71546        0.630
            13          -0.70827        0.630
            14          -0.70147        0.632
            15          -0.69505        0.632
            16          -0.68898        0.634
            17          -0.68323        0.637
            18          -0.67779        0.639
            19          -0.67263        0.646
  

### El mejor N encontrado es:

In [65]:
print(N)

40


### 5. Incluimos la lista de elementos subjetivos como features.

In [66]:
def word_feats_eS(commentWords):
    valor = valor_comentario_eS(commentWords)
    array =  dict([(word[0], word[0] in commentWords) for word in palabras_features_eS])
    array["Valor"] = valor
    return array

def valor_comentario_eS(commentWords):
    suma = 0
    for word in commentWords:
        if word in elementos_subjetivos_positivos:  
            suma += 1
        elif word in elementos_subjetivos_negativos:
            suma -= 1        
    if suma > 0:
        return 1
    elif suma < 0:
        return -1
    else:
        return 0    

def get_valor(linea):
    q = re.sub('\(\'.*\'\,','',linea)
    r = re.sub('\)','',q)
    return r

def get_palabra(linea):
    q = re.sub('\(\'','',linea)
    r = re.sub('\'\,.*\)','',q)
    return r

elementos_subjetivos_positivos = []
elementos_subjetivos_negativos = []
with codecs.open('elementos_subjetivos.txt','r',encoding='utf8') as f:
    lines = f.read().splitlines()
    for l in lines:
        if "elementoSubjetivo" in l and "%" not in l:
            p = re.sub('elementoSubjetivo','',l)
            q = re.sub('\.','',p)
            if get_valor(q) == "3":
                elementos_subjetivos_positivos.append(get_palabra(q))
            else:
                elementos_subjetivos_negativos.append(get_palabra(q))

palabras_features_eS = palabrasMasComunes.most_common(N)
array_for_training_eS = [(word_feats_eS(f[0]), classification(f[1])) for f in lista_comentarios_training_token]

### 5.1 Ahora calculamos qué tan bueno es nuestro nuevo clasificador

In [67]:
print(crossValidation(array_for_training_eS))

  ==> Training (40 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.09861        0.255
             2          -0.81266        0.589
             3          -0.79889        0.589
             4          -0.78577        0.589
             5          -0.77330        0.589
             6          -0.76147        0.593
             7          -0.75026        0.596
             8          -0.73967        0.603
             9          -0.72965        0.610
            10          -0.72019        0.617
            11          -0.71126        0.625
            12          -0.70282        0.632
            13          -0.69485        0.637
            14          -0.68732        0.641
            15          -0.68021        0.639
            16          -0.67348        0.642
            17          -0.66711        0.649
            18          -0.66107        0.653
            19          -0.65535        0.659
  